In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("diabetes.csv")

# Task-1

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [7]:
def grabColNames(dataframe,car_threshold=20,cat_threshold=10):

    cat_but_car=[col for col in dataframe.columns if (df[col].dtype=="O") and (dataframe[col]).nunique()>car_threshold]
    cat_cols=[col for col in dataframe.columns if (df[col].dtype=="O") and (col not in cat_but_car)]

    num_but_cat=[col for col in dataframe.columns if (dataframe[col].dtype !="O") and (dataframe[col].nunique()<cat_threshold)]
    num_cols=[col for col in dataframe.columns if (dataframe[col].dtype !="O") and (col not in num_but_cat)]

    cat_cols=cat_cols+num_but_cat
    print(f"Observations = {dataframe.shape[0]}\n")
    print(f"Variables = {dataframe.shape[1]}\n")
    print(f"catorical columns = {len(cat_cols)}\n")
    print(f"catorical but cardinal columns = {len(cat_but_car)}\n")
    print(f"numerical columns = {len(num_cols)}\n")
    print(f"numerical but categorical columns = {len(num_but_cat)}\n")

    return cat_cols,cat_but_car,num_cols


In [8]:
cat_cols,cat_but_car,num_cols=grabColNames(df)

Observations = 768

Variables = 9

catorical columns = 1

catorical but cardinal columns = 0

numerical columns = 8

numerical but categorical columns = 1



In [9]:
df.groupby(cat_cols).agg({"Outcome":["count","mean"]})

Outcome     
          count mean
Outcome             
0           500  0.0
1           268  1.0

In [10]:
df.groupby(["Outcome"]).agg({col:["count","mean"] for col in num_cols}).round(2).T

Outcome                              0       1
Pregnancies              count  500.00  268.00
                         mean     3.30    4.87
Glucose                  count  500.00  268.00
                         mean   109.98  141.26
BloodPressure            count  500.00  268.00
                         mean    68.18   70.82
SkinThickness            count  500.00  268.00
                         mean    19.66   22.16
Insulin                  count  500.00  268.00
                         mean    68.79  100.34
BMI                      count  500.00  268.00
                         mean    30.30   35.14
DiabetesPedigreeFunction count  500.00  268.00
                         mean     0.43    0.55
Age                      count  500.00  268.00
                         mean    31.19   37.07

In [11]:
def outlierThresholds(dataframe,column_name,q1=0.25,q3=0.75):
    quartile1=dataframe[column_name].quantile(q1)
    quartile3=dataframe[column_name].quantile(q3)
    interquantile_range=quartile3-quartile1
    low_limit=quartile1-(1.5*interquantile_range)
    up_limit=quartile3+(1.5*interquantile_range)
    return low_limit,up_limit

In [12]:
def checkOutlier(dataframe,column_name,q1=0.25,q3=0.75):
    low_limit,up_limit=outlierThresholds(dataframe,column_name,q1,q3)
    if dataframe[(dataframe[column_name]>up_limit) | (dataframe[column_name]<low_limit)].any(axis=None):
        return True
    else:
        return False

In [13]:
def replaceWithThresholds(dataframe,column_name,q1=0.25,q3=0.75):
    low_limit,up_limit=outlierThresholds(dataframe,column_name,q1,q3)
    dataframe.loc[(dataframe[column_name]>up_limit),column_name]=up_limit
    dataframe.loc[(dataframe[column_name]<low_limit),column_name]=low_limit

In [14]:
for col in num_cols:
    print(f"{col} {checkOutlier(df,col)}")

Pregnancies True
Glucose True
BloodPressure True
SkinThickness True
Insulin True
BMI True
DiabetesPedigreeFunction True
Age True


In [15]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [16]:
df.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


# Task-2

In [17]:
missing_columns=[col for col in num_cols if col!="Pregnancies" and df[col].min()==0]
missing_columns

['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

In [18]:
import numpy as np

In [19]:
for col in missing_columns:
    df[col]=df[col].apply(lambda x: np.NAN if x==0.0 else x)

In [20]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,763.0,121.686763,30.535641,44.000,99.00000,117.0000,141.00000,199.00
BloodPressure,733.0,72.405184,12.382158,24.000,64.00000,72.0000,80.00000,122.00
SkinThickness,541.0,29.153420,10.476982,7.000,22.00000,29.0000,36.00000,99.00
Insulin,394.0,155.548223,118.775855,14.000,76.25000,125.0000,190.00000,846.00
BMI,757.0,32.457464,6.924988,18.200,27.50000,32.3000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [21]:
df.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [22]:
for col in num_cols:
    print(f"{col} {checkOutlier(df,col)}")

Pregnancies True
Glucose False
BloodPressure True
SkinThickness True
Insulin True
BMI True
DiabetesPedigreeFunction True
Age True


In [23]:
for col in num_cols:
    replaceWithThresholds(df,col)

In [24]:
for col in num_cols:
    print(f"{col} {checkOutlier(df,col)}")

Pregnancies False
Glucose False
BloodPressure False
SkinThickness False
Insulin False
BMI False
DiabetesPedigreeFunction False
Age False


In [25]:
def missingValuesTable(dataframe):
    temp=pd.DataFrame()
    for idx in dataframe.isnull().sum().index:
        if dataframe.isnull().sum()[idx]!=0:
            temp[idx]=[dataframe.isnull().sum()[idx],dataframe.isnull().sum()[idx]/len(dataframe)]
    temp=temp.set_axis(["Count","Ratio"],axis="index")
    temp=temp.T
    temp["Count"]=temp["Count"].astype(int)
    print(temp)
    

In [26]:
missingValuesTable(df)

               Count     Ratio
Glucose            5  0.006510
BloodPressure     35  0.045573
SkinThickness    227  0.295573
Insulin          374  0.486979
BMI               11  0.014323


In [27]:
temp_list=[col for col in df.columns if (col not in missing_columns) and (col!="Outcome")]
while df.isnull().sum().any() and temp_list!=[]:
    for col in missing_columns:
        df[col]=df[col].fillna(df.groupby(temp_list)[col].transform("median"))
    temp_list.pop(0)

In [28]:
df.isnull().sum()

Pregnancies                  0
Glucose                      0
BloodPressure                0
SkinThickness                5
Insulin                     12
BMI                          0
DiabetesPedigreeFunction     0
Age                          0
Outcome                      0
dtype: int64

In [29]:
df[df.isnull().any(axis=1)]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
129,0.0,105.0,84.0,37.0,NaN,27.9,0.741,62.0,1
148,5.0,147.0,78.0,37.0,NaN,33.7,0.218,65.0,0
221,2.0,158.0,90.0,NaN,NaN,31.6,0.805,66.0,1
294,0.0,161.0,50.0,37.0,NaN,21.9,0.254,65.0,0
362,5.0,103.0,104.0,37.0,NaN,39.2,0.305,65.0,0
456,1.0,135.0,54.0,37.0,NaN,26.7,0.687,62.0,0
495,6.0,166.0,74.0,NaN,NaN,26.6,0.304,66.0,0
509,8.0,120.0,78.0,NaN,NaN,25.0,0.409,64.0,0
552,6.0,114.0,88.0,NaN,NaN,27.8,0.247,66.0,0
579,2.0,197.0,70.0,57.0,NaN,34.7,0.575,62.0,1


In [30]:
df=df.dropna()

In [31]:
df.columns=[col.upper() for col in df.columns]

In [32]:
df.describe()

,PREGNANCIES,GLUCOSE,BLOODPRESSURE,SKINTHICKNESS,INSULIN,BMI,DIABETESPEDIGREEFUNCTION,AGE,OUTCOME
count,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000
mean,3.828042,121.305556,72.211640,29.166005,142.812913,32.409656,0.459476,32.707011,0.349206
std,3.342015,30.337362,11.665318,8.759360,75.246201,6.697759,0.286628,11.034345,0.477035
min,0.000000,44.000000,40.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,23.000000,91.000000,27.575000,0.240750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,30.000000,122.000000,32.300000,0.374500,29.000000,0.000000
75%,6.000000,140.000000,80.000000,34.000000,180.000000,36.600000,0.626250,40.000000,1.000000
max,13.500000,199.000000,104.000000,57.000000,360.625000,50.250000,1.200000,66.500000,1.000000


In [33]:
labels=["21_30","31_40","41_50","51_60","61_70"]
bins=[20, 30, 40, 50, 60, 70]
df["AGE_CAT"]=pd.cut(df.AGE,bins=bins,labels=labels)

In [34]:
df["SKINTHICKNESS_BMI"]=df["SKINTHICKNESS"]*df["BMI"]

In [35]:
df["PREGNANCIES_BMI"]=df["PREGNANCIES"]*df["BMI"]

In [36]:
df["BLOODPRESSURE_BMI"]=df["BLOODPRESSURE"]*df["BMI"]

In [37]:
df["GLUCOSE_DIABETESPEDIGREEFUNCTION"]=df["GLUCOSE"]*df["DIABETESPEDIGREEFUNCTION"]

In [38]:
df.head()

,PREGNANCIES,GLUCOSE,BLOODPRESSURE,SKINTHICKNESS,INSULIN,BMI,DIABETESPEDIGREEFUNCTION,AGE,OUTCOME,AGE_CAT,SKINTHICKNESS_BMI,PREGNANCIES_BMI,BLOODPRESSURE_BMI,GLUCOSE_DIABETESPEDIGREEFUNCTION
0,6.0,148.0,72.0,35.0,105.5,33.6,0.627,50.0,1,41_50,1176.00,201.6,2419.2,92.796
1,1.0,85.0,66.0,29.0,183.5,26.6,0.351,31.0,0,31_40,771.40,26.6,1755.6,29.835
2,8.0,183.0,64.0,28.5,93.5,23.3,0.672,32.0,1,31_40,664.05,186.4,1491.2,122.976
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0,21_30,646.30,28.1,1854.6,14.863
4,0.0,137.0,40.0,35.0,168.0,43.1,1.200,33.0,1,31_40,1508.50,0.0,1724.0,164.400


In [39]:
df["AGE_CAT"].value_counts()

21_30    417
31_40    157
41_50    113
51_60     54
61_70     15
Name: AGE_CAT, dtype: int64

In [40]:
df=pd.get_dummies(df,columns=["AGE_CAT"],drop_first=True)

In [41]:
cat_cols,cat_but_car,num_cols=grabColNames(df,cat_threshold=20)

Observations = 756

Variables = 17

catorical columns = 6

catorical but cardinal columns = 0

numerical columns = 11

numerical but categorical columns = 6



In [42]:
cat_cols

['PREGNANCIES',
 'OUTCOME',
 'AGE_CAT_31_40',
 'AGE_CAT_41_50',
 'AGE_CAT_51_60',
 'AGE_CAT_61_70']

In [43]:
def rareAnalyser(dataframe,target_column,column_name):
    def ratio(x):
        return len(x)/len(dataframe)

    print(dataframe.groupby(column_name).agg({target_column:["count",ratio,"mean"]}).
    reset_index().sort_values(by=(f"{target_column}", 'count'),ascending=False))

In [44]:
rareAnalyser(df,"OUTCOME","PREGNANCIES")

   PREGNANCIES OUTCOME                    
                 count     ratio      mean
1          1.0     134  0.177249  0.216418
0          0.0     109  0.144180  0.339450
2          2.0     101  0.133598  0.168317
3          3.0      75  0.099206  0.360000
4          4.0      68  0.089947  0.338235
5          5.0      55  0.072751  0.381818
6          6.0      47  0.062169  0.319149
7          7.0      45  0.059524  0.555556
8          8.0      37  0.048942  0.594595
9          9.0      28  0.037037  0.642857
10        10.0      24  0.031746  0.416667
11        11.0      11  0.014550  0.636364
13        13.0      10  0.013228  0.500000
12        12.0       8  0.010582  0.500000
14        13.5       4  0.005291  1.000000


In [45]:
def rareEncoder(dataframe,rare_perc):
    temp_df=dataframe.copy()

    rare_columns=[col for col in temp_df.columns if temp_df[col].dtypes=="O"
        and (temp_df[col].value_counts()/len(temp_df)<rare_perc).any(axis=None)]

    for col in rare_columns:
        tmp=temp_df[col].value_counts()/len(temp_df)
        rare_labels=tmp[tmp<rare_perc].index
        temp_df[col]=np.where(temp_df[col].isin(rare_labels),"Rare",temp_df[col])
    
    return temp_df

In [46]:
labels=[str(i) for i in range(0,11)] + ["Rare"]
bins=[i for i in range(0,13)]
df["PREGNANCIES"]=pd.cut(df.PREGNANCIES,bins=bins,labels=labels,ordered=False)

In [47]:
df=pd.get_dummies(df,columns=["PREGNANCIES"],drop_first=True)

In [48]:
df.drop("AGE",axis=1,inplace=True)

In [49]:
cat_cols,cat_but_car,num_cols=grabColNames(df,cat_threshold=20)

Observations = 756

Variables = 26

catorical columns = 16

catorical but cardinal columns = 0

numerical columns = 10

numerical but categorical columns = 16



In [50]:
from sklearn.preprocessing import StandardScaler

In [51]:
scaler=StandardScaler()
df[num_cols]=scaler.fit_transform(df[num_cols])

In [52]:
df.head()

,GLUCOSE,BLOODPRESSURE,SKINTHICKNESS,INSULIN,BMI,DIABETESPEDIGREEFUNCTION,OUTCOME,SKINTHICKNESS_BMI,PREGNANCIES_BMI,BLOODPRESSURE_BMI,...,PREGNANCIES_2,PREGNANCIES_3,PREGNANCIES_4,PREGNANCIES_5,PREGNANCIES_6,PREGNANCIES_7,PREGNANCIES_8,PREGNANCIES_9,PREGNANCIES_10,PREGNANCIES_Rare
0,0.880502,-0.018155,0.666471,-0.496206,0.177840,0.584850,1,0.455334,0.656000,0.078997,...,0,0,0,1,0,0,0,0,0,0
1,-1.197520,-0.532840,-0.018964,0.541077,-0.867977,-0.378706,0,-0.473047,-0.838374,-0.850579,...,0,0,0,0,0,0,0,0,0,0
2,2.034959,-0.704402,-0.076084,-0.655788,-1.361006,0.741952,1,-0.719368,0.526203,-1.220952,...,0,0,0,0,0,1,0,0,0,0
3,-1.065582,-0.532840,-0.704399,-0.649139,-0.643873,-1.021078,0,-0.760097,-0.825565,-0.711899,...,0,0,0,0,0,0,0,0,0,0
4,0.517673,-2.763145,0.666471,0.334951,1.597164,2.585278,1,1.218277,-1.065519,-0.894845,...,0,0,0,0,0,0,0,0,0,0


In [53]:
y=df["OUTCOME"]
X=df.drop("OUTCOME",axis=1)

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [56]:
from sklearn.ensemble import RandomForestClassifier

In [57]:
model=RandomForestClassifier(random_state=42).fit(X_train,y_train)
y_pred=model.predict(X_test)
accuracy_score(y_pred,y_test)

0.748898678414097